In [1]:
import pandas as pd

In [2]:
keyNamePath='data/key_name.csv'
keyTestTConfigPath='data/key_test_t_config.csv'
keyTestTScenesPath='data/key_test_t_scenes.csv'
keyTestTPath='data/key_test_t.csv'

In [3]:
# keyNamePath='backup/forTest/keyName.csv'
# keyTestTConfigPath='backup/forTest/keyTestTConfig.csv'
# keyTestTScencesPath='backup/forTest/keyTestTScenes.csv'
# keyTestTPath='backup/forTest/keyTestT.csv'

In [4]:
# Read the data
keyName = pd.read_csv(keyNamePath)
# 读取keyTestTConfig.csv文件，只保留id, key, value字段，并将字段数据类型改为int64
keyTestTConfig = pd.read_csv(keyTestTConfigPath, usecols=['id', 'key', 'value'])
# 读取keyTestTScences.csv文件，只保留id, tool_config, tool_env_config, component_config, component_env_config字段
keyTestTScenes = pd.read_csv(keyTestTScenesPath, usecols=['id', 'tool_config', 'tool_env_config', 'component_config', 'component_env_config'])
# 读取keyTest.csv文件，只保留id, results, scenes_id, dimension字段
keyTestT = pd.read_csv(keyTestTPath, usecols=['id', 'results', 'scenes_id', 'dimension'])

In [5]:
# 将keyTestTScenes的id字段重命名为Scenes_id
keyTestTScenes.rename(columns={'id': 'Scenes_id'}, inplace=True)
# 将keyTestTConfig的id字段重命名为Config_id
keyTestTConfig.rename(columns={'id': 'Config_id'}, inplace=True)
# 将keyTestT的id字段重命名为Test_id
keyTestT.rename(columns={'id': 'Test_id'}, inplace=True)
# 将keyName的id字段重命名为Name_id
keyName.rename(columns={'id': 'Name_id'}, inplace=True)

In [6]:
# 将四个文件除去空值
keyName = keyName.dropna()
keyTestTConfig = keyTestTConfig.dropna()
keyTestTScences = keyTestTScenes.dropna()
keyTestT = keyTestT.dropna()

In [49]:
keyNamet=keyName
keyTestTConfigt=keyTestTConfig
keyTestTScenest=keyTestTScenes
keyTestTt=keyTestT

In [50]:
# # 将keyName, keyTestTConfig, keyTestTScences, keyTestT分别取50条数据，保存到backup文件夹下
# keyName[:50].to_csv('backup/keyName.csv', index=False)
# keyTestTConfig[:50].to_csv('backup/keyTestTConfig.csv', index=False)
# keyTestTScences[:50].to_csv('backup/keyTestTScences.csv', index=False)
# keyTestT[:50].to_csv('backup/keyTestT.csv', index=False)

In [51]:
def changeToNum(value):
    # 如果字符串中包含小数点，则去掉小数点后面的内容
    try:
        if '.' in value:
            value = value.split('.')[0]
    except:
        pass

    try:
        return int(value)
    except:
        pass

In [52]:
keyTestTt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 648279 entries, 0 to 648278
Data columns (total 4 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Test_id    648279 non-null  object
 1   results    648279 non-null  object
 2   scenes_id  648279 non-null  object
 3   dimension  648279 non-null  object
dtypes: object(4)
memory usage: 24.7+ MB


In [53]:
keyTestTConfigt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 235593 entries, 0 to 235593
Data columns (total 3 columns):
 #   Column     Non-Null Count   Dtype 
---  ------     --------------   ----- 
 0   Config_id  235593 non-null  object
 1   key        235593 non-null  object
 2   value      235593 non-null  object
dtypes: object(3)
memory usage: 7.2+ MB


In [54]:
# 将keTestT.csv文件通过scenes_id字段作为参照值，如果keyTestTScences.csv文件中id字段的值与scenes_id字段的值相同，则将keyTestTScences.csv文件中的tool_config字段的值赋值给keyTestT.csv文件中的tool_config字段
keyTestTt = pd.merge(keyTestTt, keyTestTScenest, left_on='scenes_id', right_on='Scenes_id', how='right')
# 将keyTestT中的scenes_id字段去重
# keyTestTt = keyTestTt.drop_duplicates(subset=['scenes_id'])
keyTestTt.dropna(inplace=True)

keyTestTt.drop(columns=['Scenes_id'], inplace=True)

In [55]:
configlist = ['tool_config', 'tool_env_config', 'component_config', 'component_env_config']

In [56]:
def idToValue(keyTestTt, keyTestTConfigt, configlist):
    for config in configlist:
        # 将keyTestT.csv文件通过config字段作为参照值，如果keyTestTConfig.csv文件中id字段的值与config字段的值相同，则将keyTestTConfig.csv文件中的value字段的值赋值给keyTestT.csv文件中的config字段
        keyTestTt = pd.merge(keyTestTt, keyTestTConfigt[['Config_id','value']], left_on=config, right_on='Config_id', how='left')
        keyTestTt.dropna(inplace=True)
        # 将tool_config字段的值替换为config
        keyTestTt[config] = keyTestTt['value']
        # 将value, Config_id字段删除
        keyTestTt.drop(['value', 'Config_id'], axis=1, inplace=True)
    return keyTestTt

In [57]:
# keyTestTt = idToValue(keyTestTt, keyTestTConfigt, configlist)

In [58]:
keyTestTt.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28397 entries, 0 to 34195
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Test_id               28397 non-null  object
 1   results               28397 non-null  object
 2   scenes_id             28397 non-null  object
 3   dimension             28397 non-null  object
 4   tool_config           28397 non-null  object
 5   tool_env_config       28397 non-null  object
 6   component_config      28397 non-null  object
 7   component_env_config  28397 non-null  object
dtypes: object(8)
memory usage: 1.9+ MB


In [59]:
def test(keyTestTt, keyTestTConfigt, configlist):
    for config in configlist:
        keyTestTt = pd.merge(keyTestTt, keyTestTConfigt[['Config_id','value']], left_on=config, right_on='Config_id', how='left')
        # keyTestTt.dropna(inplace=True)
        # 将tool_config字段的值替换为value字段的值
        keyTestTt[config] = keyTestTt['value']
        # 将value, Config_id字段删除
        keyTestTt.drop(['value', 'Config_id'], axis=1, inplace=True)
    return keyTestTt

In [60]:
smalltest=test(keyTestTt, keyTestTConfigt, configlist)

In [61]:
smalltest.head()

,Test_id,results,scenes_id,dimension,tool_config,tool_env_config,component_config,component_env_config
0,2385,"{""#vray_render_time"": 145}",72,"{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_...",NaN,"{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...",NaN
1,2932,"{""#vray_render_time"": 146}",72,"{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_...",NaN,"{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...",NaN
2,2939,"{""#vray_render_time"": 149}",72,"{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_...",NaN,"{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...",NaN
3,2394,"{""#vray_render_time"": 144}",72,"{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_...",NaN,"{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...",NaN
4,2403,"{""#vray_render_time"": 148}",72,"{""cvm_cpu"": ""8"", ""platform"": ""default"", ""test_...",NaN,"{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...",NaN


In [62]:
smalltest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28397 entries, 0 to 28396
Data columns (total 8 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   Test_id               28397 non-null  object
 1   results               28397 non-null  object
 2   scenes_id             28397 non-null  object
 3   dimension             28397 non-null  object
 4   tool_config           1947 non-null   object
 5   tool_env_config       28397 non-null  object
 6   component_config      28397 non-null  object
 7   component_env_config  0 non-null      object
dtypes: object(8)
memory usage: 1.9+ MB


In [63]:
smalltest=smalltest[['Test_id','results','tool_env_config','component_config']]

In [64]:
smalltest.head()

,Test_id,results,tool_env_config,component_config
0,2385,"{""#vray_render_time"": 145}","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""..."
1,2932,"{""#vray_render_time"": 146}","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""..."
2,2939,"{""#vray_render_time"": 149}","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""..."
3,2394,"{""#vray_render_time"": 144}","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""..."
4,2403,"{""#vray_render_time"": 148}","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""...","{""name"": ""cvm"", ""configs"": {""cpu"": {""Flags"": ""..."


In [65]:
smalltest.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 28397 entries, 0 to 28396
Data columns (total 4 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Test_id           28397 non-null  object
 1   results           28397 non-null  object
 2   tool_env_config   28397 non-null  object
 3   component_config  28397 non-null  object
dtypes: object(4)
memory usage: 1.1+ MB


In [66]:
# 将keyTestT文件中的tool_config字段的值通过与keyTestConfig文件的Config_id字段作为参照值，如果keyTestConfig文件中的id字段的值与tool_config字段的值相同，则将keyTestConfig文件中的value字段的值赋值给keyTestT文件中的tool_config字段
# keyTestTt = pd.merge(keyTestTt, keyTestTConfigt[['Config_id','value']], left_on='tool_env_config', right_on='Config_id', how='left')
# keyTestTt.dropna(inplace=True)
# # 将tool_config字段的值替换为value字段的值
# keyTestTt['tool_env_config'] = keyTestTt['value']
# # 将value, Config_id字段删除
# keyTestTt.drop(['value', 'Config_id'], axis=1, inplace=True)
# keyTestTt.head()

In [67]:
# 将keyTestT.csv保存为test.csv文件
# keyTestT.to_csv('test.csv', index=False)